<a href="https://colab.research.google.com/github/mwai21/Python-Baby-Steps/blob/master/meetup187_tim_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MeetUp 187 - Beginners' Python and Machine Learning - 26 Jul 2023 - pandas

Learning objectives:
- Introduction to data handling with pandas


Links:
- Colab: https://colab.research.google.com/drive/1FODx3cH8h3DOx6mRlzDgKrWk0rMZklfd
- Youtube: https://youtu.be/r058lI5Yr8E
- Meetup:  https://www.meetup.com/beginners-python-machine-learning/events/294698691/
- Github:  https://github.com/timcu/bpaml-sessions/tree/master/online

@author D Tim Cummings

Today we are introducing pandas and some of its features useful for data science. We will collect data from an online API and store in a DataFrame. Then we will perform filtering, converting, calculating and pivoting the data.

- https://pandas.pydata.org

# Using Google Colab / Jupyter Notebooks / IPython

- type into a cell
- press `<shift><enter>` to execute the cell
- cells can be python code or markdown text or input fields
- use ? or Help menu for help
- see Help menu > Keyboard shortcuts


In [ ]:
# How to determine pandas version from the command line
!pip list | grep pandas

In [ ]:
# Start using pandas by importing it. To save time typing 'pd' instead of 'pandas' use an import alias
import pandas as pd

In [ ]:
# Let's get some data from https://www.data.brisbane.qld.gov.au/data/dataset/public-art and store it in a DataFrame
# Use the pandas method read_csv to read data from URL or from a file
# Good if you know the encoding. I had to try a few before this worked.
# Default encoding is 'utf-8'. I also tried 'latin1' which didn't show quotes correctly
url_art = 'https://www.data.brisbane.qld.gov.au/data/dataset/1e11bcdd-fab1-4ec5-b671-396fd1e6dd70/resource/3c972b8e-9340-4b6d-8c7b-2ed988aa3343/download/public-art-open-data-2023-03-14.csv'
df_art = pd.read_csv(url_art, encoding='cp1252')
# Look at the data
df_art

In [ ]:
# In pandas, DataFrames are a two dimensional array of data.
# Series are a one dimensional array of data. Each column in a DataFrame can be used a Series
# Use [] notation with name of column to isolate just one column
s = df_art['Item_title']
print(f"df_art is a {type(df_art)} and s is a {type(s)}")

In [ ]:
# A series has an index as well as a column of values
s

In [ ]:
# To get a sub DataFrame use a list of column names
df_art[['Item_title', 'Latitude', 'Longitude']]

In [ ]:
# Click on wizard to view dataframe in colab,
# Click chart icon to show example charts
# Select chart Longitude vs Latitude
# Add cell
# Edit supplied code to plot Latitude vs Longitude and change scatter_plot_size to 7.5

In [ ]:
# Modify code to show just Latitude (y) versus Longitude (x) with scatter_plot_size=5

In [ ]:
import numpy as np

def scatter_plots(df, colname_pairs, scatter_plot_size=2.5, size=8, alpha=.6):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * scatter_plot_size, scatter_plot_size))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    ax.scatter(df[x_colname], df[y_colname], s=size, alpha=alpha)
    plt.xlabel(x_colname)
    plt.ylabel(y_colname)
    ax.spines[['top', 'right',]].set_visible(False)
    # Add some titles if they are provided
    if "Item_title" in df:
        for idx in df.index:
            if idx % 5 == 0:
                # We can refer to individual items in a DataFrame using .at[row, col]
                x = df.at[idx, x_colname]
                y = df.at[idx, y_colname]
                t = df.at[idx, "Item_title"]
                if y < -27.49 or y > -27.45:
                    # show text next to some data points
                    ax.text(x + 0.001, y + 0.001, t)
  plt.tight_layout()
  return plt

chart = scatter_plots(df_art, *[[['Longitude', 'Latitude']]], scatter_plot_size=5, **{})
chart;

In [ ]:
# Using .at to access a single data element
df_art.at[4, "Item_title"]

In [ ]:
# Methods to access part of DataFrame via named index and columns
# df.at[row, col] => access one element
# df.loc[rows, cols] => access multiple elements

# Methods to access part of DataFrame via numbered index and columns
# df.iat[irow, icol] => access one element by zero based integer indexes
# df.iloc[irows, icols] => access multiple elements by zero based integer indexes

In [ ]:
# Using .loc to access a single data element
df_art.loc[4, "Item_title"]

In [ ]:
# .loc can be used with a sub DataFrame of rows just like we retrieved a sub DataFrame of columns before
# Notice how slice includes ending row which is unusual in Python
df_art.loc[2:5]

In [ ]:
# Can also slice on columns, but have to provide a slice for rows. : = all rows
# Notice how we have to provide a space at end of Material because that is in original data
df_art.loc[:, 'Artist':'Material ']

In [ ]:
# List all column names
df_art.columns

In [ ]:
# We can rename columns. Notice how we are using `inplace=True` so we are editing the DataFrame rather than returning a new one.
new_names = {n: n.strip() for n in df_art.columns}
print(f"{new_names=}")
df_art.rename(columns=new_names, inplace=True)
df_art.columns

In [ ]:
# Save csv file from BCC Open Data to see what caused the spaces at end of column names
from urllib.request import urlopen
with urlopen(url_art) as file_obj:
    with open('public_art.csv', 'wb') as pa_csv:
        pa_csv.write(file_obj.read())
# View the file using the operating system command line rather than python code
!head public_art.csv

In [ ]:
# Advanced - Find which lines are not utf-8
with open('public_art.csv', 'rb') as pa_csv:
    for i, line in enumerate(pa_csv):
        try:
            s = line.decode('utf-8')
        except UnicodeDecodeError:
            print(i, ' '*10, line)
            # latin1 and iso-8859-1 are the same
            # https://docs.python.org/3.9/library/codecs.html#standard-encodings
            for encoding in ['cp1252', 'latin1', 'mac-roman']:
                s = line.decode(encoding)
                print(i, f"{encoding:12}", s.strip())

In [ ]:
df_art

In [ ]:
# Can use a list of bools on rows or columns to specify which ones to include
df_art.loc[:, [True, False, True, False, False, False, False, False]]

In [ ]:
# Can create a list of bools using an expression
df_art.columns.str.contains("_")

In [ ]:
# Put them together
df_art.loc[:, df_art.columns.str.contains("_")]

In [ ]:
# Could also use earlier expression df[[column names, ...]]
df_art[df_art.columns[df_art.columns.str.contains("_")]]

In [ ]:
# More common to filter this way on rows
# Find art installed last year
df_art.loc[df_art['Installed']==2022]
# Alternatively
# df_art[df_art['Installed']==2022]


In [ ]:
# Look at traffic data
# https://www.data.brisbane.qld.gov.au/data/dataset/traffic-management-key-corridor-monthly-performance-report/resource/70da5292-87a1-4fd4-8a35-b0a723566884
# Remove bom=True from CSV URL https://www.data.brisbane.qld.gov.au/data/datastore/dump/70da5292-87a1-4fd4-8a35-b0a723566884?bom=True
url_tfc = 'https://www.data.brisbane.qld.gov.au/data/datastore/dump/70da5292-87a1-4fd4-8a35-b0a723566884'
with urlopen(url_tfc) as file_obj:
    with open('traffic.csv', 'wb') as tfc_csv:
        tfc_csv.write(file_obj.read())
# View the file using the operating system command line rather than python code
!head traffic.csv

In [ ]:
# Read csv from file
df_tfc = pd.read_csv('traffic.csv')
df_tfc

In [ ]:
# Perform arithmetical operation on a column and store the result
df_tfc['Volume per hour average'] = df_tfc['Average Weekday Daily Traffic (Veh/day)'] / 24
df_tfc

In [ ]:
# Calculations can be more complicated, although not every calculation can work
# Operators (+,-,*,/) are overriden so this works.
vol_am = 'Volume per hour AM Peak'
vol_pm = 'Volume per hour PM Peak'
tt_am = 'Average TT AM Peak (seconds)'
tt_pm = 'Average TT PM Peak (seconds)'
tt = 'Average TT Peak (seconds)'
df_tfc[tt] = (df_tfc[tt_am] * df_tfc[vol_am] + df_tfc[tt_pm] * df_tfc[vol_pm]) / (df_tfc[vol_am] + df_tfc[vol_pm])
df_tfc

In [ ]:
df_tfc[df_tfc['Month']=='May']

In [ ]:
# Check data for Site ID 2 in May 2023
avg_tt = (1865 * 1182 + 1899 * 1023) / (1865 + 1899)
avg_tt

In [ ]:
# Check using assert and filters
assert avg_tt == df_tfc.loc[(df_tfc['Year'] == 2023) & (df_tfc['Month'] == 'May') & (df_tfc['Site ID'] == 2), 'Average TT Peak (seconds)'].values[0]
df_tfc.loc[(df_tfc['Year'] == 2023) & (df_tfc['Month'] == 'May') & (df_tfc['Site ID'] == 2), 'Average TT Peak (seconds)']

In [ ]:
# Check using query instead of filters
assert avg_tt == df_tfc.query('Year == 2023 & Month == "May" & `Site ID` == 2')['Average TT Peak (seconds)'].values[0]
df_tfc.query('Year == 2023 & Month == "May" & `Site ID` == 2')['Average TT Peak (seconds)']

In [ ]:
# To call functions use numpy functions rather than Python functions
import math
# The following won't work
# df_tfc['sin_id'] = math.sin(df_tfc['_id'] * math.pi / 180)

In [ ]:
# Use sin from numpy which can work on ndarrays
import numpy as np
df_tfc['sin_id'] = np.sin(df_tfc['_id'] * math.pi / 180)
df_tfc

In [ ]:
# For datetime use pandas.to_datetime
df_tfc['Year Month'] = pd.to_datetime(df_tfc['Year'].astype(str) + df_tfc['Month'], format=("%Y%b"))
df_tfc

In [ ]:
# Make the Site names more interesting than Site ID and take a portion of the DataFrame
df_tfc['Site Name'] = 'Site ' + df_tfc['Site ID'].astype(str).str.zfill(2)
df_vol = df_tfc[['Site Name', 'Year Month', 'Average Weekday Daily Traffic (Veh/day)']]
df_vol.head(30)

In [ ]:
# Now we can pivot data
df_pivot = df_vol.pivot(index='Year Month', columns='Site Name', values='Average Weekday Daily Traffic (Veh/day)')
df_pivot